# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['kapur_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
11.279659916105501  |  12.518934499283429  |  11.59679837404883
11.254557060310738  |  4.712693007684774  |  11.19350821801332
11.32495530892971  |  4.712693007684774  |  11.880550004892484
11.060119713989918  |  4.712693007684774  |  11.684262182491915
10.96865289831506  |  4.712693007684774  |  11.726356722442976
11.202223654184074  |  4.712693007684774  |  11.931342449952648
11.43636565721689  |  11.990693441612008  |  11.907388268288987
11.15135535984368  |  4.712693007684774  |  11.884300747984495
10.120687127243336  |  4.712693007684774  |  11.802201428086406
11.042666156761403  |  12.10082511603414  |  11.93362381426096
11.16721788795115  |  4.712693007684774  |  11.17851213894302
11.134426411638044  |  4.712693007684774  |  11.639053695120417
10.68774897867571  |  4.712693007684774  |  11.590955481257803
11.028095307046309  |  4.712693007684774  |  12.132991923328564
11.271207147365146  |  4.712693007684774  |  11.864365150529473
10.661248013

4.830145490563806  |  12.673173025306422  |  11.753615412599425
4.830145490563806  |  12.673111851906794  |  11.996042852241906
4.830145490563806  |  5.2637859707782555  |  11.774436386050752
4.830145490563806  |  5.2637859707782555  |  12.055254836926238
4.830145490563806  |  11.772976559647066  |  11.96911787506815
11.32742454658423  |  12.41805506200985  |  12.060233717647717
4.830145490563806  |  5.2637859707782555  |  11.114891731489617
4.830145490563806  |  12.10036479652583  |  11.507255617782988
4.830145490563806  |  5.2637859707782555  |  12.00994778331198
4.830145490563806  |  5.2637859707782555  |  11.870893995031404
4.830145490563806  |  5.2637859707782555  |  11.62342595902548
4.830145490563806  |  12.698959273000352  |  12.031032211490256
4.830145490563806  |  11.698444666666706  |  11.756467869460497
4.830145490563806  |  12.375087967552947  |  11.874331993338512
4.830145490563806  |  12.347564016811404  |  11.740952953668323
4.830145490563806  |  12.01897381198696  |  1

5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  11.833442337115619
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.215005569729137  |  11.689776188615337  |  5.215005569729137
5.215005569729137  |  5.215005569729137  |  5.215005569729137
5.2150

5.373370941037026  |  12.282877494238399  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
10.042607919933538  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  12.279983483343717  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.373370941037026  |  5.180881180377972  |  12.716263347516822
5.373370941037026  |  5.180881180377972  |  5.341330396962399
5.37

4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  5.196006870846936  |  5.086887216666606
12.278584359830333  |  5.196006870846936  |  5.086887216666606
4.892444611886998  |  11.420810326786054  |  5.086887216666606
4.8924

5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  10.616754142966872
5.000115318505147  |  5.297622559553731  |  11.565627371773328
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  11.895548643216763
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  11.702834899050362
5.000115318505147  |  11.9423194786398  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  11.857979182836843
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.000115318505147  |  5.297622559553731  |  5.068561029488975
5.00

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,kapur_entropy,max,"[96, 161]","[7, 15]","[107, 163]",11.066663,5.461772,11.851974,...,"[[107, 255, 255, 255, 255, 255, 255, 255, 255,...","[11.279659916105501, 11.254557060310738, 11.32...","[12.518934499283429, 4.712693007684774, 4.7126...","[11.59679837404883, 11.19350821801332, 11.8805...","[5.312496900558472, 6.004200220108032, 5.67947...","[3129.033014933268, 4149.7346026102705, 5139.7...","[55.93776018874252, 64.41843371745598, 71.6920...","[13.176702153862436, 11.950600386877586, 11.02...","[0.7193201261991956, 0.640837609550924, 0.6267...","[0.923758094595045, 0.9090108339557803, 0.8906..."
1,AirplaneF16.tiff,3,kapur_entropy,max,"[77, 124, 176]","[60, 115, 174]","[82, 145, 183]",13.757440,14.541663,14.583841,...,"[[145, 255, 255, 255, 255, 255, 255, 255, 255,...","[13.646355362483249, 14.266765705152789, 14.11...","[14.929613728605977, 15.38179898545975, 14.506...","[15.046426795145948, 14.205037680227164, 13.96...","[12.083884954452515, 8.696452379226685, 9.9148...","[1736.2746569315593, 1743.0918032328289, 2624....","[41.668629170295, 41.75035093544519, 51.229478...","[15.734619346387069, 15.717601002137318, 13.94...","[0.7317210762891787, 0.7540783408693622, 0.699...","[0.9537656869966972, 0.9538368679491483, 0.935..."
2,AirplaneF16.tiff,4,kapur_entropy,max,"[61, 99, 149, 190]","[55, 103, 155, 194]","[65, 114, 148, 190]",16.057377,17.110534,16.633018,...,"[[114, 255, 255, 255, 255, 255, 255, 255, 255,...","[15.69850514950954, 15.71985519814967, 16.3298...","[16.976853273881176, 15.102699683807359, 17.38...","[16.53907432164429, 16.13536850540936, 16.1479...","[10.002529859542847, 7.914614677429199, 7.7626...","[837.7097244262695, 2508.369916280111, 1156.10...","[28.94321551635667, 50.08362922432949, 34.0015...","[18.899868039546316, 14.136887773724318, 17.50...","[0.7170435135855429, 0.756971928935973, 0.7089...","[0.9704207830305408, 0.9387262908899632, 0.962..."
3,AirplaneF16.tiff,5,kapur_entropy,max,"[55, 98, 133, 166, 198]","[42, 78, 110, 154, 194]","[57, 93, 127, 161, 200]",17.628903,19.175201,18.690463,...,"[[127, 255, 200, 255, 255, 255, 200, 200, 200,...","[19.617856389041016, 17.955200905445746, 18.45...","[17.889096843316437, 20.558054555538934, 20.04...","[19.958254648782308, 19.212084090257264, 18.62...","[8.995712041854858, 7.8406689167022705, 7.8344...","[2193.48464457194, 1411.7340126037598, 1566.49...","[46.834652177334895, 37.5730490192606, 39.5789...","[14.719457623827093, 16.63327482664966, 16.181...","[0.744562297704782, 0.7127028816441743, 0.7798...","[0.9501020629693229, 0.9641519575361895, 0.959..."
4,Lena.png,2,kapur_entropy,max,"[122, 183]","[47, 95]","[12, 14]",5.046721,9.471223,11.746814,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[4.830145490563806, 4.830145490563806, 4.83014...","[12.673173025306422, 12.673111851906794, 5.263...","[11.753615412599425, 11.996042852241906, 11.77...","[9.62273907661438, 8.641907453536987, 9.254782...","[9257.71044031779, 9142.93359629313, 17585.324...","[96.21699662906647, 95.61868853050187, 132.609...","[8.465767679314656, 8.519947953019388, 5.67929...","[0.514370651376642, 0.5288310410261157, 0.4128...","[0.7478116442750036, 0.7492577652230817, 0.627..."
5,Lena.png,3,kapur_entropy,max,"[116, 156, 202]","[56, 120, 177]","[48, 82, 107]",9.489548,14.748951,14.404125,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[4.830145490563806, 4.830145490563806, 12.9389...","[14.897625524276883, 14.77944769946816, 15.375...","[14.388823731606912, 14.861764103597666, 14.69...","[9.655999422073364, 9.49759578704834, 7.677339...","[9101.777140299479, 8851.567180633545, 1908.26...","[95.4032344331128, 94.0827677135061, 43.683692...","[8.539541633959642, 8.660601910010874, 15.3244...","[0.528

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/SMA/SMA_kapur_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/SMA/SMA_kapur_RGB_dict.pkl', 'wb'))